In [1]:
import src.main as lc
import torch, os

# Filepaths
HHD = "/volumes/Ultra Touch"
MLP_LOC = HHD + "/MLP/LC_test"
COMPRESSED_SAVELOC = HHD + "/MLP/Compressed"
DECOMPRESSED_SAVELOC = HHD + "/MLP/Decompressed_test"

# Base dictionary (for decompressor to understand structure of the model)
BASE_DICT = torch.load(MLP_LOC + "/frame0.tar", map_location = torch.device('cpu'))["network_fn_state_dict"]

# 1. Compression of MLP model set

In [2]:
lc.compress_set(MLP_LOC, COMPRESSED_SAVELOC)

Loading: /volumes/Ultra Touch/MLP/LC_test/frame0.tar
Delta Compression on: frame1.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame1.tar
Delta Compression on: frame2.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame2.tar
Delta Compression on: frame3.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame3.tar
Delta Compression on: frame4.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame4.tar
Saving Compressed Format: frame0.tar
Saving Compressed Format: frame1.tar
Saving Compressed Format: frame2.tar
Saving Compressed Format: frame3.tar
Saving Compressed Format: frame4.tar


# 2. Decompression from Compressed Set

In [3]:
lc.load_compressed_set(COMPRESSED_SAVELOC, DECOMPRESSED_SAVELOC, BASE_DICT)

Decompressing for: compressed_frame0.pt
Decompressing for: compressed_frame1.pt
Decompressing for: compressed_frame2.pt
Decompressing for: compressed_frame3.pt
Decompressing for: compressed_frame4.pt
Saving Decompressed Model at: decompressed_frame0.pt
Saving Decompressed Model at: decompressed_frame1.pt
Saving Decompressed Model at: decompressed_frame2.pt
Saving Decompressed Model at: decompressed_frame3.pt
Saving Decompressed Model at: decompressed_frame4.pt


# 3. Testing Random Weights in decompressed MLP

In [8]:
frame_no = 3
original = torch.load(MLP_LOC + "/frame{}.tar".format(frame_no), map_location = torch.device('cpu'))["network_fn_state_dict"]
compressed = lc.read_decompressed_state_dict(DECOMPRESSED_SAVELOC + "/decompressed_frame{}.pt".format(frame_no))

In [19]:
original['pts_linears.1.weight'][2][10:20]

tensor([-0.1713, -0.1642, -0.0814, -0.1256, -0.0692, -0.1202, -0.0272,  0.0780,
         0.1156, -0.0341])

In [20]:
compressed['pts_linears.1.weight'][2][10:20]

tensor([-0.1419, -0.1430, -0.0833, -0.1399, -0.0644, -0.1347, -0.0267,  0.0944,
         0.1023, -0.0290])

In [14]:
frame_no = 1
original = torch.load(MLP_LOC + "/frame{}.tar".format(frame_no), map_location = torch.device('cpu'))["network_fn_state_dict"]
compressed = lc.read_decompressed_state_dict(DECOMPRESSED_SAVELOC + "/decompressed_frame{}.pt".format(frame_no))

In [17]:
original['pts_linears.1.weight'][4][5:15]

tensor([-0.2480, -0.2322,  0.0630, -0.1238, -0.0873,  0.1134,  0.1539,  0.0085,
         0.0692, -0.0862])

In [18]:
compressed['pts_linears.1.weight'][4][5:15]

tensor([-0.1702, -0.2176,  0.0619, -0.1270, -0.0907,  0.1337,  0.1742,  0.0024,
         0.0702, -0.0826])

# 4. Compression Ratio

In [7]:
uncompressed_size = sum(os.path.getsize(MLP_LOC + "/" + f) for f in os.listdir(MLP_LOC))
compressed_size = sum(os.path.getsize(COMPRESSED_SAVELOC + "/" + f) for f in os.listdir(COMPRESSED_SAVELOC))
print("Compression Ratio:")
print(round((uncompressed_size - compressed_size) / uncompressed_size, 5))

Compression Ratio:
0.94242
